In [ ]:
!wget https://www.manythings.org/anki/tam-eng.zip

--2020-10-31 15:26:35--  https://www.manythings.org/anki/tam-eng.zip
Resolving www.manythings.org (www.manythings.org)... 104.24.108.196, 172.67.173.198, 104.24.109.196, ...
Connecting to www.manythings.org (www.manythings.org)|104.24.108.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9823 (9.6K) [application/zip]
Saving to: ‘tam-eng.zip’

tam-eng.zip         100%[===================>]   9.59K  --.-KB/s    in 0s      

2020-10-31 15:26:35 (85.3 MB/s) - ‘tam-eng.zip’ saved [9823/9823]



In [ ]:
!pwd

/content


In [ ]:
import os
os.listdir('/')

['media',
 'var',
 'dev',
 'tmp',
 'sys',
 'home',
 'sbin',
 'mnt',
 'lib64',
 'run',
 'bin',
 'usr',
 'opt',
 'boot',
 'srv',
 'etc',
 'root',
 'proc',
 'lib',
 'index.html',
 'content',
 'tam-eng.zip',
 'data',
 '.dockerenv',
 'datalab',
 'tools',
 'swift',
 'tensorflow-1.15.2',
 'lib32']

In [ ]:
import subprocess
import argparse
import os

import pandas as pd

# parser = argparse.ArgumentParser()
# parser.add_argument('-s', help='ISO 639-3 code for a source language')
# parser.add_argument('-t', help='ISO 639-3 code for a target language')
# parser.add_argument('-data_dir', default='./data', help='Data directory')


if __name__ == '__main__':
    # args = parser.parse_args()
    source_lang = 'eng'
    target_lang = 'tam'
    data_dir = '/content/data/'

    # download if sentences are not here
    sentences_path = os.path.join(data_dir, 'sentences.csv')
    if not os.path.isfile(sentences_path):
        if not os.path.isfile(os.path.join(data_dir, 'sentences.tar.bz2')):
            subprocess.run(
                "wget https://downloads.tatoeba.org/exports/sentences.tar.bz2 -P " + data_dir,
                shell=True)

        subprocess.run(
            "tar xvjC {0} -f {0}/sentences.tar.bz2".format(data_dir), shell=True)

    # download if links are not here
    links_path = os.path.join(data_dir, 'links.csv')
    if not os.path.isfile(links_path):
        if not os.path.isfile(os.path.join(data_dir, 'links.tar.bz2')):
            subprocess.run(
                "wget https://downloads.tatoeba.org/exports/links.tar.bz2 -P " + data_dir,
                shell=True)

        subprocess.run("tar xvjC {0} -f {0}/links.tar.bz2".format(data_dir), shell=True)

    # read all data
    sentences = pd.read_csv(sentences_path, names=['id', 'lang', 'text'], header=None, delimiter='\t')
    links = pd.read_csv(links_path, names=['sent_id', 'tran_id'], header=None, delimiter='\t')

    # extract source - target connected
    source_sentences = sentences[sentences.lang == source_lang]
    source_sentences = source_sentences.merge(links, left_on='id', right_on='sent_id')
    target_sentences = sentences[sentences.lang == target_lang]

    bilang_sentences = source_sentences.merge(target_sentences, left_on='tran_id',
                                              right_on='id',
                                              suffixes=[source_lang, target_lang])
    bilang_sentences = bilang_sentences[['text' + source_lang, 'text' + target_lang]]

    # save results
    file_name = os.path.join(data_dir, '{source}-{target}.csv'.format(source=source_lang, target=target_lang))
    bilang_sentences.to_csv(file_name, index=False, sep='\t')

In [ ]:
import pandas as pd
import os

source_lang = 'eng'
target_lang = 'tam'
data_dir = '/content/data/'

os.chdir('../')
corpus = pd.read_csv(os.path.join(data_dir, '{}-{}.csv'.format(source_lang, target_lang)), delimiter='\t')

In [ ]:
corpus

,texteng,texttam
0,Most people think I'm crazy.,நிறைய மக்கள் நான் பைத்தியம் என்று எண்ணுகிறார்கள்
1,I love you.,நான் உன்னை காதலிக்கிறேன்.
2,Where are you?,நீ எங்கே இருக்கிறாய்?
3,When did you come to Japan?,நீ எப்பொழுது ஜப்பான் வந்தாய்?
4,When is your birthday?,உங்கள் பிறந்த நாள் எப்போது ?
...,...,...
312,I will sleep.,நான் தூங்குவேன்.
313,They told me why they needed my help.,அவர்கள் எதனால் என் உதவி தேவைப்பட்டது என்று கூற...
314,Tom drank with us until after midnight.,டாம் நள்ளிரவு வரை எங்களுடன் குடித்தார்.
315,Tom's car was manufactured in the 1980s.,டாமின் கார் 1980 களில் தயாரிக்கப்பட்டது.


In [ ]:
SOS_token = '<start>'
EOS_token = '<end>'
UNK_token = '<unk>'
PAD_token = '<pad>'

SOS_idx = 0
EOS_idx = 1
UNK_idx = 2
PAD_idx = 3

class Vocab:
    def __init__(self):
        self.index2word = {
            SOS_idx: SOS_token,
            EOS_idx: EOS_token,
            UNK_idx: UNK_token,
            PAD_idx: PAD_token
        }
        self.word2index = {v: k for k, v in self.index2word.items()}

    def index_words(self, words):
        for word in words:
            self.index_word(word)

    def index_word(self, word):
        if word not in self.word2index:
            n_words = len(self)
            self.word2index[word] = n_words
            self.index2word[n_words] = word

    def __len__(self):
        assert len(self.index2word) == len(self.word2index)
        return len(self.index2word)

    def unidex_words(self, indices):
        return [self.index2word[i] for i in indices]

    def to_file(self, filename):
        values = [w for w, k in sorted(list(self.word2index.items())[5:])]
        with open(filename, 'w') as f:
            f.write('\n'.join(values))

    @classmethod
    def from_file(cls, filename):
        vocab = Vocab()
        with open(filename, 'r') as f:
            words = [l.strip() for l in f.readlines()]
            vocab.index_words(words)

In [ ]:
!pip install inltk

In [ ]:
!pip install torch==1.3.1+cu100 torchvision==0.4.2+cu100 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [ ]:
from inltk.inltk import setup
setup('ta')

RuntimeError: ignored

Done!


In [ ]:
from inltk.inltk import tokenize

def tamil_tokenize(text):
  return tokenize(text, "ta")

In [ ]:
import nltk
# from tokenize_uk import tokenize_words

import pandas as pd

max_length = 10
min_word_count = 1

tokenizers = {
    'tam': tamil_tokenize,
    'eng': nltk.tokenize.WordPunctTokenizer().tokenize
}

def preprocess_corpus(sents, tokenizer, min_word_count):
    n_words = {}

    sents_tokenized = []
    for sent in sents:
        sent_tokenized = [w.lower() for w in tokenizer(sent)]

        sents_tokenized.append(sent_tokenized)

        for word in sent_tokenized:
            if word in n_words:
                n_words[word] += 1
            else:
                n_words[word] = 1

    for i, sent_tokenized in enumerate(sents_tokenized):
        sent_tokenized = [t if n_words[t] >= min_word_count else UNK_token for t in sent_tokenized]
        sents_tokenized[i] = sent_tokenized

    return sents_tokenized

def read_vocab(sents):
    vocab = Vocab()
    for sent in sents:
        vocab.index_words(sent)

    return vocab

source_sents = preprocess_corpus(corpus['text' + source_lang], tokenizers[source_lang], min_word_count)
target_sents = preprocess_corpus(corpus['text' + target_lang], tokenizers[target_lang], min_word_count)

# Using set to remove duplicates
source_sents, target_sents = zip(
    *sorted({(tuple(s), tuple(t)) for s, t in zip(source_sents, target_sents)
              if len(s) <= max_length and len(t) <= max_length})
)

source_vocab = read_vocab(source_sents)
target_vocab = read_vocab(target_sents)

target_vocab.to_file(os.path.join(data_dir, '{}.vocab.txt'.format(target_lang)))
source_vocab.to_file(os.path.join(data_dir, '{}.vocab.txt'.format(source_lang)))

print('Corpus length: {}\nSource vocabulary size: {}\nTarget vocabulary size: {}'.format(
    len(source_sents), len(source_vocab.word2index), len(target_vocab.word2index)
))
examples = list(zip(source_sents, target_sents))[80:90]
for source, target in examples:
    print('Source: "{}", target: "{}"'.format(' '.join(source), ' '.join(target)))

Corpus length: 243
Source vocabulary size: 391
Target vocabulary size: 648
Source: "i am not a guide .", target: "▁நான் ▁ஒரு ▁வழி ▁காட்டி ▁இல்லை"
Source: "i am not being guided .", target: "▁நான் ▁வழி ▁காட்ட ப் ▁பட்டு க் ▁கொண்டு ▁இருக்க ▁ வில்லை"
Source: "i am not guided .", target: "▁நான் ▁வழி ▁காட்ட ப் ▁பட ▁ வில்லை"
Source: "i am sleeping .", target: "▁நான் ▁தூ ங்க ுகிற ேன் ."
Source: "i arrived ahead of the others .", target: "▁மற்றவர் களுக்கு ▁முன்ன ே ▁நான் ▁வந்த ேன்"
Source: "i ate too much .", target: "▁நான் ▁நிறைய ▁சாப்பிட ் டே ன்"
Source: "i came to buy vegetables .", target: "▁நான் ▁காய்கறி ▁வாங்க ▁கடை க்குப் ▁போன ேன்"
Source: "i didn ' t inform them .", target: "▁நான் ▁அவர் களிடம் ▁சொல்ல வில்லை ."
Source: "i didn ' t let them know .", target: "▁நான் ▁அவர் களிடம் ▁தெரிவிக்க வில்லை ."
Source: "i didn ' t tell them .", target: "▁நான் ▁அவர் களிடம் ▁சொல்ல வில்லை ."


In [ ]:
source_to_target = {}
for source, target in zip(source_sents, target_sents):
    if source in source_to_target:
        source_to_target[source].append(target)
    else:
        source_to_target[source] = [target]

source_sents, target_sents = zip(*source_to_target.items())
len(source_sents)

242

In [ ]:
import numpy as np

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

source_length = len(source_sents)
inidices = np.random.permutation(source_length)

training_indices = inidices[:int(source_length*0.8)]
dev_indices = inidices[int(source_length*0.8):int(source_length*0.86)]
test_indices = inidices[int(source_length*0.86):]

training_source = [source_sents[i] for i in training_indices]
dev_source = [source_sents[i] for i in dev_indices]
test_source = [source_sents[i] for i in test_indices]

training_target = [target_sents[i] for i in training_indices]
dev_target = [target_sents[i] for i in dev_indices]
test_target = [target_sents[i] for i in test_indices]

# Unwrap training examples
training_t = []
training_s = []
for source, tt in zip(training_source, training_target):
    for target in tt:
        training_t.append(target)
        training_s.append(source)

training_source = training_s
training_target = training_t

In [ ]:
import torch

def indexes_from_sentence(vocab, sentence):
    return [vocab.word2index[word] for word in sentence]

def tensor_from_sentence(vocab, sentence, max_seq_length):
    indexes = indexes_from_sentence(vocab, sentence)
    indexes.append(EOS_idx)
    indexes.insert(0, SOS_idx)
    # we need to have all sequences the same length to process them in batches
    if len(indexes) < max_seq_length:
        indexes += [PAD_idx] * (max_seq_length - len(indexes))
    tensor = torch.LongTensor(indexes)
    return tensor

def tensors_from_pair(source_sent, target_sent, max_seq_length):
    source_tensor = tensor_from_sentence(source_vocab, source_sent, max_seq_length).unsqueeze(1)
    target_tensor = tensor_from_sentence(target_vocab, target_sent, max_seq_length).unsqueeze(1)
    return (source_tensor, target_tensor)

max_seq_length = max_length + 2  # 2 for EOS_token and SOS_token

training = []
for source_sent, target_sent in zip(training_source, training_target):
    training.append(tensors_from_pair(source_sent, target_sent, max_seq_length))

x_training, y_training = zip(*training)
x_training = torch.transpose(torch.cat(x_training, dim=-1), 1, 0)
y_training = torch.transpose(torch.cat(y_training, dim=-1), 1, 0)
torch.save(x_training, os.path.join(data_dir, 'x_training.bin'))
torch.save(y_training, os.path.join(data_dir, 'y_training.bin'))

x_development = []
for source_sent in dev_source:
    tensor = tensor_from_sentence(source_vocab, source_sent, max_seq_length).unsqueeze(1)
    x_development.append(tensor)

x_development = torch.transpose(torch.cat(x_development, dim=-1), 1, 0)
torch.save(x_development, os.path.join(data_dir, 'x_development.bin'))

x_test = []
for source_sent in test_source:
    tensor = tensor_from_sentence(source_vocab, source_sent, max_seq_length).unsqueeze(1)
    x_test.append(tensor)

x_test = torch.transpose(torch.cat(x_test, dim=-1), 1, 0)
torch.save(x_test, os.path.join(data_dir, 'x_test.bin'))

USE_CUDA = True
if USE_CUDA:
    x_training = x_training.cuda()
    y_training = y_training.cuda()
    x_development = x_development.cuda()
    x_test = x_test.cuda()

NameError: ignored